<a href="https://colab.research.google.com/github/yutao-data/Sepsis_Gene_KnowledgeGraph_Ontology/blob/main/Sepsis_SPARQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
!wget -v https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

if os.path.exists('spark-3.5.1-bin-hadoop3.tgz'):
    print("Spark tarball downloaded successfully.")
    !tar -xzf spark-3.5.1-bin-hadoop3.tgz -C /usr/local/

    if os.path.exists('/usr/local/spark-3.5.1-bin-hadoop3'):
        !rm -rf /usr/local/spark
        !mv /usr/local/spark-3.5.1-bin-hadoop3 /usr/local/spark
        print("Spark moved to /usr/local/spark")
    else:
        print("Failed to extract Spark tarball.")
else:
    print("Failed to download the Spark tarball. Please check the URL or your internet connection.")

!wget -v https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz

if os.path.exists('hadoop-3.3.6.tar.gz'):
    print("Hadoop tarball downloaded successfully.")
    !tar -xzf hadoop-3.3.6.tar.gz -C /usr/local/

    if os.path.exists('/usr/local/hadoop-3.3.6'):
        !rm -rf /usr/local/hadoop
        !mv /usr/local/hadoop-3.3.6 /usr/local/hadoop
        print("Hadoop moved to /usr/local/hadoop")
    else:
        print("Failed to extract Hadoop tarball.")
else:
    print("Failed to download the Hadoop tarball. Please check the URL or your internet connection.")

java_home_path = os.popen('readlink -f /usr/bin/java | sed "s:bin/java::"').read().strip()

if java_home_path:
    with open('/usr/local/spark/conf/spark-env.sh', 'a') as f:
        f.write(f"export JAVA_HOME={java_home_path}\n")
    with open('/usr/local/hadoop/etc/hadoop/hadoop-env.sh', 'a') as f:
        f.write(f"export JAVA_HOME={java_home_path}\n")
    print("JAVA_HOME added to /usr/local/spark/conf/spark-env.sh and /usr/local/hadoop/etc/hadoop/hadoop-env.sh")
else:
    print("Failed to find JAVA_HOME.")

os.environ['JAVA_HOME'] = java_home_path
os.environ['SPARK_HOME'] = '/usr/local/spark'
os.environ['HADOOP_HOME'] = '/usr/local/hadoop'
os.environ['PATH'] = f"/usr/local/spark/bin:/usr/local/hadoop/bin:{os.environ['PATH']}"

!pip install -q findspark
import findspark
findspark.init('/usr/local/spark')

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("Colab Spark Test").getOrCreate()
print("Spark session created successfully")
spark.stop()

!hadoop version

--2024-07-28 20:17:50--  https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400446614 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.1-bin-hadoop3.tgz.12’

spark-3.5.1-bin-had 100%[===================>] 381.90M  24.2MB/s    in 17s     

2024-07-28 20:18:08 (21.9 MB/s) - ‘spark-3.5.1-bin-hadoop3.tgz.12’ saved [400446614/400446614]

Spark tarball downloaded successfully.
Spark moved to /usr/local/spark
--2024-07-28 20:18:14--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
H

In [21]:
!hadoop version

!spark-shell --version

Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /usr/local/hadoop/share/hadoop/common/hadoop-common-3.3.6.jar
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.1
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.23
Branch HEAD
Compiled by user heartsavior on 2024-02-15T11:24:58Z
Revision fd86f85e181fc2dc0f50a096855acf83a6cc5d9c
Url https://github.com/apache/spark
Type --help for more information.


In [4]:
# Update package list and install opam
!apt-get update -qq > /dev/null
!apt-get install -qq opam

# Initialize opam and install packages
!opam init --disable-sandboxing --yes
!opam switch create 4.13.1
!eval $(opam env)
!opam install menhir yojson --yes

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
W: https://repo.scala-sbt.org/scalasbt/debian/dists/all/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: https://repo.scala-sbt.org/scalasbt/debian/Release.gpg: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
[WARNING] Running as root is not recommended

<><> Required setup - please read <><><><><><><><><><><><><><><><><><><><><><><>

  In normal operation, opam only alters files within ~/.opam.

  However, to best integrate with your system, some environment variables
  should be set. If you allow it to, this initialisation step will update
  your bash configuration by adding the following line to ~/.profile:

    test -r /root/.opam/opam-

In [5]:
# Install sbt
!echo "deb https://repo.scala-sbt.org/scalasbt/debian all main" | tee /etc/apt/sources.list.d/sbt.list
!echo "deb https://repo.scala-sbt.org/scalasbt/debian /" | tee /etc/apt/sources.list.d/sbt_old.list
!curl -sL "https://keyserver.ubuntu.com/pks/lookup?op=get&search=0x99E82A75642AC823" | apt-key add
!apt-get update
!apt-get install sbt

# Install menhir and yojson via opam
!opam install menhir yojson ocamlfind --yes
!eval $(opam env)

deb https://repo.scala-sbt.org/scalasbt/debian all main
deb https://repo.scala-sbt.org/scalasbt/debian /
OK
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 https://scala.jfrog.io/artifactory/debian all InRelease
Ign:10 https://scala.jfrog.io/artifactory/debian  InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://scala.jfrog.io/artifactory/debian  Release
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:14 https://ppa.

In [6]:
# Remove the existing sparqlgx directory
!rm -rf sparqlgx

# Clone the SPARQLGX repository
!git clone https://github.com/tyrex-team/sparqlgx.git

# Navigate to the SPARQLGX directory and update the Makefile to include --infer flag
!sed -i 's/menhir -v Parser.mly/menhir --infer -v Parser.mly/' sparqlgx/Makefile

# Verify the Makefile change
!grep 'menhir --infer -v Parser.mly' sparqlgx/Makefile

# Ensure the opam environment is loaded within the script and then run the setup scripts
!cd sparqlgx && eval $(opam env) && bash dependencies.sh && bash compile.sh

Cloning into 'sparqlgx'...
remote: Enumerating objects: 2536, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2536 (delta 0), reused 1 (delta 0), pack-reused 2530
Receiving objects: 100% (2536/2536), 2.33 MiB | 4.57 MiB/s, done.
Resolving deltas: 100% (1287/1287), done.
sed: can't read sparqlgx/Makefile: No such file or directory
grep: sparqlgx/Makefile: No such file or directory
[WARNING] Running as root is not recommended
Checking dependencies...
[Success]	sbt exists.
[Success]	menhir exists.
[Success]	ocamlc exists.
[Success]	ocamldep exists.
[Success]	ocamllex exists.
[Success]	ocamlopt exists.
[Success]	hadoop exists.
[Success]	spark-submit exists.
[Success]	ocamlfind exists.
[Success]	yojson exists.

====== SPARQLGX compilation chain ======
---- Generation of build.sbt ----
build.sbt in src/load-src/
---- Load Source Compilation ----
[info] Updated file /content/sparqlgx/src/load-src/project/build.properties: set sbt.

In [7]:
!opam install menhir --yes
!eval $(opam env)
!which menhir

[WARNING] Running as root is not recommended
[NOTE] Package menhir is already installed (current version is 20240715).
[WARNING] Running as root is not recommended


In [8]:
!hadoop fs -mkdir -p /user/root/input
!hadoop fs -put /content/drive/MyDrive/stagelisn2024-main/Owl/enriched_ppio_ontology_with_kegg_Pathways.owl /user/root/input/

In [9]:
!cd sparqlgx && bash bin/sparqlgx.sh load myDatabase /user/root/input/enriched_ppio_ontology_with_kegg_Pathways.owl

24/07/28 20:21:20 INFO SparkContext: Running Spark version 3.5.1
24/07/28 20:21:20 INFO SparkContext: OS info Linux, 6.1.85+, amd64
24/07/28 20:21:20 INFO SparkContext: Java version 11.0.23
24/07/28 20:21:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/28 20:21:21 INFO ResourceUtils: ==============================================================
24/07/28 20:21:21 INFO ResourceUtils: No custom resources configured for spark.driver.
24/07/28 20:21:21 INFO ResourceUtils: ==============================================================
24/07/28 20:21:21 INFO SparkContext: Submitted application: Simple Application
24/07/28 20:21:21 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name:

In [10]:
!ocaml -version
!opam list

The OCaml toplevel, version 4.13.1
[WARNING] Running as root is not recommended
# Packages matching: installed
# Name            # Installed # Synopsis
base-bigarray     base
base-threads      base
base-unix         base
dune              3.16.0      Fast, portable, and opinionated build system
host-arch-x86_64  1           OCaml on amd64 (64-bit)
host-system-other 1           OCaml on an unidentified system
menhir            20240715    An LR(1) parser generator
menhirCST         20240715    Runtime support library for parsers generated by Menhir
menhirLib         20240715    Runtime support library for parsers generated by Menhir
menhirSdk         20240715    Compile-time library for auxiliary tools related to Menhir
ocaml             4.13.1      The OCaml compiler (virtual package)
ocaml-config      2           OCaml Switch Configuration
ocaml-system      4.13.1      The OCaml compiler (system version, from outside of opam)
ocamlfind         1.9.6       A library manager for OCaml
s

In [16]:
import os
import subprocess

def check_command(command, expected_output=None, is_error_expected=False):
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output = result.stderr.decode().strip() if is_error_expected else result.stdout.decode().strip()
        if expected_output and expected_output not in output:
            print(f"Command '{command}' executed but did not find expected output: '{expected_output}'")
            print(f"Output: {output}")
            return False
        print(f"Command '{command}' executed successfully.")
        print(f"Output: {output}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error executing command '{command}': {e.stderr.decode().strip()}")
        return False

def check_java():
    return check_command("java -version", "openjdk", is_error_expected=True)

def check_sbt():
    return check_command("sbt --version", "sbt version")

def check_spark():
    spark_home = os.getenv('SPARK_HOME')
    if not spark_home:
        print("SPARK_HOME environment variable is not set.")
        return False
    return check_command(f"{spark_home}/bin/spark-submit --version", "version", is_error_expected=True)

def check_sparqlgx():
    if not os.path.exists("sparqlgx"):
        print("sparqlgx directory does not exist.")
        return False
    return check_command("cd sparqlgx && sbt clean package", "success")

def main():
    print("Checking Java installation...")
    java_installed = check_java()

    print("Checking SBT installation...")
    sbt_installed = check_sbt()

    print("Checking Spark installation...")
    spark_installed = check_spark()

    print("Checking SPARQLGX installation...")
    sparqlgx_installed = check_sparqlgx()

    if java_installed and sbt_installed and spark_installed and sparqlgx_installed:
        print("All required environments are correctly installed.")
    else:
        print("Some environments are not correctly installed. Please check the above logs for details.")

if __name__ == "__main__":
    main()

Checking Java installation...
Command 'java -version' executed successfully.
Output: openjdk version "11.0.23" 2024-04-16
OpenJDK Runtime Environment (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)
Checking SBT installation...
Command 'sbt --version' executed successfully.
Output: sbt version in this project: 1.10.1
sbt script version: 1.10.1
Checking Spark installation...
Command '/usr/local/spark/bin/spark-submit --version' executed successfully.
Output: Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.1
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.23
Branch HEAD
Compiled by user heartsavior on 2024-02-15T11:24:58Z
Revision fd86f85e181fc2dc0f50a096855acf83a6cc5d9c
Url https://github.com/apache/spark
Type --help for more information.
Checking SPARQLGX installa

In [27]:
os.environ['HDFS_NAMENODE_USER'] = 'root'
os.environ['HDFS_DATANODE_USER'] = 'root'
os.environ['HDFS_SECONDARYNAMENODE_USER'] = 'root'

!echo 'Y' | hdfs namenode -format

!$HADOOP_HOME/sbin/start-dfs.sh

!jps

!hadoop fs -rm -r /rdf_data/enriched_ppio_ontology_with_kegg_Pathways.nt

!hadoop fs -mkdir -p /rdf_data
!hadoop fs -copyFromLocal /content/enriched_ppio_ontology_with_kegg_Pathways.nt /rdf_data/enriched_ppio_ontology_with_kegg_Pathways.nt

!cd sparqlgx && bash bin/sparqlgx.sh light-load myDatabase /rdf_data/enriched_ppio_ontology_with_kegg_Pathways.nt
!cd sparqlgx && bash bin/sparqlgx.sh generate-stat myDatabase /rdf_data/enriched_ppio_ontology_with_kegg_Pathways.nt
!cd sparqlgx && bash bin/sparqlgx.sh load myDatabase /rdf_data/enriched_ppio_ontology_with_kegg_Pathways.nt

sparql_query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
LIMIT 10
"""

with open("sparqlgx/query.rq", "w") as file:
    file.write(sparql_query)

!cd sparqlgx && bash bin/sparqlgx.sh query myDatabase sparqlgx/query.rq

with open("sparqlgx/query_output.txt", "r") as file:
    print(file.read())

2024-07-28 20:47:29,507 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = 4ae5c1b8e490/172.28.0.12
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.3.6
STARTUP_MSG:   classpath = /usr/local/hadoop/etc/hadoop:/usr/local/hadoop/share/hadoop/common/lib/netty-transport-sctp-4.1.89.Final.jar:/usr/local/hadoop/share/hadoop/common/lib/netty-transport-4.1.89.Final.jar:/usr/local/hadoop/share/hadoop/common/lib/commons-beanutils-1.9.4.jar:/usr/local/hadoop/share/hadoop/common/lib/jersey-server-1.19.4.jar:/usr/local/hadoop/share/hadoop/common/lib/netty-codec-stomp-4.1.89.Final.jar:/usr/local/hadoop/share/hadoop/common/lib/jsp-api-2.1.jar:/usr/local/hadoop/share/hadoop/common/lib/netty-transport-native-kqueue-4.1.89.Final-osx-x86_64.jar:/usr/local/hadoop/share/hadoop/common/lib/jetty-http-9.4.51.v20230217.jar:/usr/local/hadoop/share/hadoop/common/lib/netty-codec-xml-4.1.89.Final.jar:/us

FileNotFoundError: [Errno 2] No such file or directory: 'sparqlgx/query_output.txt'